In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.stem import WordNetLemmatizer

import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
#import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline


# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [2]:
df = pd.read_csv('/Users/maurelco/Developer/Python/Projet 4/data/Cleaned/df_process_text_3.csv')

In [3]:
df

,Title,Body,Tags,_clean_tags,_len_body,Body_2,_len_body_2,Body_3,Title_2
0,giving unix process exclusive rw access directory,way sandbox linux process certain directory gi...,linux ubuntu process sandbox selinux,"['linux', 'ubuntu', 'process', 'sandbox', 'sel...",526,sandbox linux process certain directory give p...,462,sandbox linux process certain directory proces...,giving unix process exclusive rw access directory
1,automatic repaint minimizing window,jframe two panel one panel draw line working m...,java graphics jframe jpanel paint,"['java', 'graphics', 'jframe', 'jpanel', 'paint']",2969,jframe two panel panel draw line working minim...,2855,jframe panel panel draw line minimized window ...,automatic repaint minimizing window
2,man-in-the-middle attack security threat ssh a...,expert network security pardon question smart ...,security ssh ssh-keys openssh man-in-the-middle,"['security', 'ssh', 'ssh-keys', 'openssh', 'ma...",447,expert network security pardon smart automatin...,414,expert network security pardon smart automatin...,man-in-the-middle attack security threat ssh a...
3,managing data access simple winforms app,simple winforms data entry app us sqlite alway...,c# winforms sqlite datatable sqlconnection,"['c#', 'winforms', 'sqlite', 'datatable', 'sql...",2537,simple winforms entry app u sqlite always sing...,2382,winforms entry app sqlite always single-user a...,managing data access winforms app
4,render basic html view,basic node.js app trying get ground using expr...,javascript html node.js mongodb express,"['javascript', 'html', 'node.js', 'mongodb', '...",335,basic node.js app get ground express framework...,292,basic node.js app get ground express framework...,render basic html view
...,...,...,...,...,...,...,...,...,...
49995,bypass vertica error execution time exceeded r...,using ssis tool ole db downloading data vertic...,sql-server ssas oledb sql-server-data-tools ve...,"['sql-server', 'ssas', 'oledb', 'sql-server-da...",206,ssis tool ole db downloading vertica database ...,186,ssis tool ole db downloading vertica database ...,bypass vertica error execution time exceeded r...
49996,conflicting conditional operation currently pr...,using f uploading file already created bucket ...,python amazon-web-services amazon-s boto pytho...,"['python', 'amazon-web-services', 'amazon-s', ...",301,f uploading file already created bucket deleti...,268,uploading file bucket deleting bucket executio...,conflicting conditional operation progress bucket
49997,problem lr_find pytorch fastai course,following jupyter notebook course hit upon err...,python machine-learning deep-learning pytorch ...,"['python', 'machine-learning', 'deep-learning'...",908,jupyter notebook course hit upon line cnn_lear...,781,jupyter notebook course hit upon line cnn_lear...,problem lr_find pytorch fastai course
49998,jsonpatch escape slash jsonpatch+json,json wanted update field process my-process po...,java json rest json-patch http-patch,"['java', 'json', 'rest', 'json-patch', 'http-p...",645,json wanted update field process my-process po...,566,json wanted field process my-process pod some-...,jsonpatch escape slash jsonpatch+json


In [4]:
df['Title_2'][37847] = 'nan'
df['Title_2'][37847]

/var/folders/lf/xpbzxzn13ss6xxtstbmn6t540000gn/T/ipykernel_12121/970875707.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Title_2'][37847] = 'nan'


'nan'

In [5]:
df['Title_2'][14558] = 'call'

/var/folders/lf/xpbzxzn13ss6xxtstbmn6t540000gn/T/ipykernel_12121/3876688444.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Title_2'][14558] = 'call'


In [7]:
df['Body_3'][19855] = 'correct'

/var/folders/lf/xpbzxzn13ss6xxtstbmn6t540000gn/T/ipykernel_12121/1708886119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Body_3'][19855] = 'correct'


In [9]:
df['body_and_title_2'] = df['Body_3'] + " " +df['Title_2']
df['body_and_title_2'].iloc[0]

'sandbox linux process certain directory process exclusive rw access dir temporary directory e.g python scripting tool directory limiting much functionality process access directory except superusers course sandbox web service basically allows user arbitrary code authorization software process linux user user harm system temporary private directory file protected user webservice giving unix process exclusive rw access directory'

In [8]:
df.isna().sum()

Title          1
Body           0
Tags           0
_clean_tags    0
_len_body      0
Body_2         0
_len_body_2    0
Body_3         0
Title_2        0
dtype: int64

### 2. Separating Train /Test

In [10]:
train, test = train_test_split(df, test_size=0.3, random_state=42)

In [11]:
train.shape

(35000, 10)

In [12]:
test.shape

(15000, 10)

In [13]:
list_body_train = list(train['Body_3'])
list_body_train[:2]

['apology sound noobish lot experience web development flash aside find best solution project sql echo php file echo border tr objectname url description picture price location category priority tr row mysqli_fetch_array echo tr echo row echo row objectname echo row url echo row description echo row picture echo row price echo row location echo row category echo row priority echo row echo row echo tr echo pull whole sql user echo row separately separate vizualize echo tile isotope jquery plug http isotope.metafizzy .co best dynamically generate divs based row fill row sorry broad',
 'hibernate.properties file successfully sessionfactory configuration object configuration configuration configuration .addannotatedclass weatherresponse.class sessionfactory sessionfactory configuration.buildsessionfactory hibernate.properties typesafe config library pa configs environment deleted hibernate.properties set programmatically configuration configuration configuration .addannotatedclass weatherr

In [14]:
list_title_train = list(train['Title_2'])
list_title_train[:2]

['create divs tile sql table', 'hibernate programmatic configuration']

In [15]:
list_title_body_train = list(train['body_and_title_2'])
list_title_body_train[:2]

['apology sound noobish lot experience web development flash aside find best solution project sql echo php file echo border tr objectname url description picture price location category priority tr row mysqli_fetch_array echo tr echo row echo row objectname echo row url echo row description echo row picture echo row price echo row location echo row category echo row priority echo row echo row echo tr echo pull whole sql user echo row separately separate vizualize echo tile isotope jquery plug http isotope.metafizzy .co best dynamically generate divs based row fill row sorry broad create divs tile sql table',
 'hibernate.properties file successfully sessionfactory configuration object configuration configuration configuration .addannotatedclass weatherresponse.class sessionfactory sessionfactory configuration.buildsessionfactory hibernate.properties typesafe config library pa configs environment deleted hibernate.properties set programmatically configuration configuration configuration 

In [16]:
list_body_test = list(test['Body_3'])
list_body_test[:2]

['small writing unit django is_valid seen code day reading google yet find looking code forms.py test_forms.py forms.py class dataselectform forms.form #these init funct result_type forms.choicefield widget forms.select attrs class field-long band_selection forms.multiplechoicefield widget forms.selectmultiple attrs class multiselect field-long title forms.charfield widget forms.hiddeninput description forms.charfield widget forms.hiddeninput __init__ self result_list band_list args kwargs super dataselectform self .__init__ args kwargs result_list band_list self.fields result_type forms.choicefield choice result_list widget forms.select attrs class field-long self.fields band_selection forms.multiplechoicefield widget forms.selectmultiple attrs class multiselect field-long choice band_list test_forms.py test_data_select_form self resulttype.objects .all results_value results_value.append result.result_type band satelliteband.objects .all bands_value band band bands_value.append band.b

In [17]:
list_title_test = list(test['Title_2'])
list_title_test[:2]

['django form unittest choicefield multiplechoicefield failing is_valid',
 'eclipse rcp application multi-window design multiple monitor']

In [18]:
list_title_body_test = list(test['body_and_title_2'])
list_title_body_test[:2]

['small writing unit django is_valid seen code day reading google yet find looking code forms.py test_forms.py forms.py class dataselectform forms.form #these init funct result_type forms.choicefield widget forms.select attrs class field-long band_selection forms.multiplechoicefield widget forms.selectmultiple attrs class multiselect field-long title forms.charfield widget forms.hiddeninput description forms.charfield widget forms.hiddeninput __init__ self result_list band_list args kwargs super dataselectform self .__init__ args kwargs result_list band_list self.fields result_type forms.choicefield choice result_list widget forms.select attrs class field-long self.fields band_selection forms.multiplechoicefield widget forms.selectmultiple attrs class multiselect field-long choice band_list test_forms.py test_data_select_form self resulttype.objects .all results_value results_value.append result.result_type band satelliteband.objects .all bands_value band band bands_value.append band.b

### 1. Approche FIT on TITTLE + BODY and TRANSFORM on TITLE + BODY

In [19]:
vectorizer_1 = CountVectorizer(
    lowercase=False,
    analyzer='word',
    min_df=3,
    max_df= 0.90,
    token_pattern= r'[a-zA_\-+#]*\.?[a-zA_\+#]+',
    )
matrix_train_1 = vectorizer_1.fit_transform(list_title_body_train)

In [20]:
matrix_test_1 = vectorizer_1.transform(list_title_body_test)

In [21]:
vectorizer_1.get_feature_names

<bound method CountVectorizer.get_feature_names of CountVectorizer(lowercase=False, max_df=0.9, min_df=3,
                token_pattern='[a-zA_\\-+#]*\\.?[a-zA_\\+#]+')>

In [22]:
count_array_1 = matrix_test_1.toarray()
df_1 = pd.DataFrame(data=count_array_1,columns = vectorizer_1.get_feature_names())
df_1

/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,#,##,###,####,#####,######,########,#########,##########,################,...,zx,zxing,zxing.mobile,zxing.net,zxn,zy,zygoteinit.java,zz,zza,zzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
df_1['java']

0        0
1        0
2        0
3        0
4        0
        ..
14995    0
14996    0
14997    0
14998    0
14999    1
Name: java, Length: 15000, dtype: int64

In [ ]:
vectorizer_test = CountVectorizer(
    lowercase=False,
    analyzer='word',
    min_df=3,
    max_df= 0.90,
    token_pattern= r'[a-zA]+',
    )
matrix_train_test = vectorizer_test.fit_transform(list_title_body_train)

In [ ]:
matrix_test_tst = vectorizer_test.transform(list_title_body_test)

In [ ]:
count_array_1_test = matrix_test_tst.toarray()
df_1 = pd.DataFrame(data=count_array_1_test,columns = vectorizer_test.get_feature_names())
df_1

### 2. Approche FIT on TITTLE and TRANSFORM on TITLE

In [22]:
vectorizer_2 = CountVectorizer(
    lowercase=False,
    analyzer='word',
    min_df=3,
    max_df= 0.9,
    token_pattern= r'[a-zA_\-+#]*\.?[a-zA_\+#]+',
    )
matrix_2_train = vectorizer_2.fit_transform(list_title_train)

In [23]:
matrix_2_test = vectorizer_2.transform(list_title_test)

In [24]:
count_array_2 = matrix_2_test.toarray()
df_2 = pd.DataFrame(data=count_array_2,columns = vectorizer_2.get_feature_names())
df_2

/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,#,#define,#pragma,++i,-a,-bit,-byte,-c,-d,-digit,...,zlib,zombie,zone,zoneddatetime,zoom,zooming,zsh,zurb,zuul,zxing
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3. Approche fit TITTLE and transform TITLE + BODY

In [25]:
vectorizer_3 = CountVectorizer(
    lowercase=False,
    analyzer='word',
    min_df=3,
    token_pattern= r'[a-zA_\-+#]*\.?[a-zA_\+#]+',
    )
matrix_3_train = vectorizer_3.fit_transform(list_title_train)

In [26]:
matrix_3_test = vectorizer_3.transform(list_title_body_test)

In [27]:
count_array_3 = matrix_3_test.toarray()
df_3 = pd.DataFrame(data=count_array_3,columns = vectorizer_3.get_feature_names())
df_3

/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,#,#define,#pragma,++i,-a,-bit,-byte,-c,-d,-digit,...,zlib,zombie,zone,zoneddatetime,zoom,zooming,zsh,zurb,zuul,zxing
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
14997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 4. LDA with SKLEARN

### 1. Approach 1 - LDA with text processing keeping non-words characters (i.e c++)

In [118]:
# Build LDA Model
lda_model_sklearn_6 = LatentDirichletAllocation(n_components=6,
                                      # Number of topics
                                      max_iter=10,
                                      # Max learning iterations
                                      learning_method='online',
                                      random_state=100,
                                      # Random state
                                      batch_size=500,
                                      # n docs in each learning iter
                                      evaluate_every = -1,
                                      # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,
                                      # Use all available CPUs
                                      )
lda_output_sklearn_6 = lda_model_sklearn_6.fit(matrix_train_1)
print(lda_output_sklearn_6)  # Model attributes

LatentDirichletAllocation(batch_size=500, learning_method='online',
                          n_components=6, n_jobs=-1, random_state=100)


In [247]:
lda_model_sklearn_6.components_

array([[5.98199942e+00, 2.96744039e+02, 5.87900638e+01, ...,
        3.97806820e+01, 1.66675554e-01, 1.65428043e+01],
       [1.67400280e-01, 1.68070961e-01, 1.68922544e-01, ...,
        1.77235598e-01, 1.67390890e-01, 1.76310364e+01],
       [1.68796796e-01, 1.67453402e-01, 1.71274784e-01, ...,
        2.74912175e+00, 1.66723570e-01, 1.66699447e-01],
       [1.73866108e+01, 1.68351685e-01, 1.68841811e-01, ...,
        1.70285191e-01, 1.66730030e-01, 2.23079588e-01],
       [2.77991406e+00, 1.67515474e-01, 1.67044410e-01, ...,
        1.71595159e-01, 4.12341093e+00, 1.67709748e-01],
       [2.52264598e-01, 5.04035388e+01, 2.71803137e+01, ...,
        1.67095223e-01, 1.66816785e-01, 1.67696018e-01]])

In [154]:
lda_sklearn_output = lda_model_sklearn_6.transform(matrix_test_1)
lda_sklearn_output

array([[0.18646123, 0.00168814, 0.17286089, 0.61810247, 0.00169301,
        0.01919426],
       [0.0036744 , 0.00364108, 0.16815333, 0.00368525, 0.00366618,
        0.81717976],
       [0.00100622, 0.0009991 , 0.03053318, 0.13023375, 0.01553792,
        0.82168984],
       ...,
       [0.22345754, 0.00127507, 0.4683252 , 0.30437265, 0.00128705,
        0.00128249],
       [0.1160545 , 0.00321071, 0.00324201, 0.00324544, 0.2425108 ,
        0.63173654],
       [0.24863167, 0.0054019 , 0.00542202, 0.00543843, 0.00546319,
        0.72964279]])

In [ ]:
tf_feature_names = vectorizer_1.get_feature_names_out()

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 15
display_topics(lda_model, tf_feature_names, no_top_words)

In [ ]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(matrix_train))
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(matrix_train))
# See model parameters
print(lda_model.get_params())

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, matrix_train, vectorizer_1, mds='tsne')
panel

In [ ]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = tf_feature_names
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

## LDA with Regex text processing avoiding all non-words

In [ ]:
# Build LDA Model
lda_model_test = LatentDirichletAllocation(n_components=15,
                                      # Number of topics
                                      max_iter=10,
                                      # Max learning iterations
                                      learning_method='online',
                                      random_state=100,
                                      # Random state
                                      batch_size=500,
                                      # n docs in each learning iter
                                      evaluate_every = -1,
                                      # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,
                                      # Use all available CPUs
                                      )
lda_output_test = lda_model_test.fit(matrix_train_test)
print(lda_model)  # Model attributes

In [ ]:
tf_feature_names_test = vectorizer_test.get_feature_names_out()

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 15
display_topics(lda_model_test, tf_feature_names_test, no_top_words)

In [ ]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model_test.score(matrix_train_test))
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model_test.perplexity(matrix_train_test))
# See model parameters
print(lda_model_test.get_params())

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model_test, matrix_train_test, vectorizer_test)
panel

## LDA with GENSIM

In [23]:
df

,Title,Body,Tags,_clean_tags,_len_body,Body_2,_len_body_2,Body_3,Title_2,body_and_title_2
0,giving unix process exclusive rw access directory,way sandbox linux process certain directory gi...,linux ubuntu process sandbox selinux,"['linux', 'ubuntu', 'process', 'sandbox', 'sel...",526,sandbox linux process certain directory give p...,462,sandbox linux process certain directory proces...,giving unix process exclusive rw access directory,sandbox linux process certain directory proces...
1,automatic repaint minimizing window,jframe two panel one panel draw line working m...,java graphics jframe jpanel paint,"['java', 'graphics', 'jframe', 'jpanel', 'paint']",2969,jframe two panel panel draw line working minim...,2855,jframe panel panel draw line minimized window ...,automatic repaint minimizing window,jframe panel panel draw line minimized window ...
2,man-in-the-middle attack security threat ssh a...,expert network security pardon question smart ...,security ssh ssh-keys openssh man-in-the-middle,"['security', 'ssh', 'ssh-keys', 'openssh', 'ma...",447,expert network security pardon smart automatin...,414,expert network security pardon smart automatin...,man-in-the-middle attack security threat ssh a...,expert network security pardon smart automatin...
3,managing data access simple winforms app,simple winforms data entry app us sqlite alway...,c# winforms sqlite datatable sqlconnection,"['c#', 'winforms', 'sqlite', 'datatable', 'sql...",2537,simple winforms entry app u sqlite always sing...,2382,winforms entry app sqlite always single-user a...,managing data access winforms app,winforms entry app sqlite always single-user a...
4,render basic html view,basic node.js app trying get ground using expr...,javascript html node.js mongodb express,"['javascript', 'html', 'node.js', 'mongodb', '...",335,basic node.js app get ground express framework...,292,basic node.js app get ground express framework...,render basic html view,basic node.js app get ground express framework...
...,...,...,...,...,...,...,...,...,...,...
49995,bypass vertica error execution time exceeded r...,using ssis tool ole db downloading data vertic...,sql-server ssas oledb sql-server-data-tools ve...,"['sql-server', 'ssas', 'oledb', 'sql-server-da...",206,ssis tool ole db downloading vertica database ...,186,ssis tool ole db downloading vertica database ...,bypass vertica error execution time exceeded r...,ssis tool ole db downloading vertica database ...
49996,conflicting conditional operation currently pr...,using f uploading file already created bucket ...,python amazon-web-services amazon-s boto pytho...,"['python', 'amazon-web-services', 'amazon-s', ...",301,f uploading file already created bucket deleti...,268,uploading file bucket deleting bucket executio...,conflicting conditional operation progress bucket,uploading file bucket deleting bucket executio...
49997,problem lr_find pytorch fastai course,following jupyter notebook course hit upon err...,python machine-learning deep-learning pytorch ...,"['python', 'machine-learning', 'deep-learning'...",908,jupyter notebook course hit upon line cnn_lear...,781,jupyter notebook course hit upon line cnn_lear...,problem lr_find pytorch fastai course,jupyter notebook course hit upon line cnn_lear...
49998,jsonpatch escape slash jsonpatch+json,json wanted update field process my-process po...,java json rest json-patch http-patch,"['java', 'json', 'rest', 'json-patch', 'http-p...",645,json wanted update field process my-process po...,566,json wanted field process my-process pod some-...,jsonpatch escape slash jsonpatch+json,json wanted field process my-process pod some-...


In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [24]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words_train = list(sent_to_words(list_title_body_train))

print(data_words_train[:1])

[['apology', 'sound', 'noobish', 'lot', 'experience', 'web', 'development', 'flash', 'aside', 'find', 'best', 'solution', 'project', 'sql', 'echo', 'php', 'file', 'echo', 'border', 'tr', 'objectname', 'url', 'description', 'picture', 'price', 'location', 'category', 'priority', 'tr', 'row', 'echo', 'tr', 'echo', 'row', 'echo', 'row', 'objectname', 'echo', 'row', 'url', 'echo', 'row', 'description', 'echo', 'row', 'picture', 'echo', 'row', 'price', 'echo', 'row', 'location', 'echo', 'row', 'category', 'echo', 'row', 'priority', 'echo', 'row', 'echo', 'row', 'echo', 'tr', 'echo', 'pull', 'whole', 'sql', 'user', 'echo', 'row', 'separately', 'separate', 'vizualize', 'echo', 'tile', 'isotope', 'jquery', 'plug', 'http', 'isotope', 'metafizzy', 'co', 'best', 'dynamically', 'generate', 'divs', 'based', 'row', 'fill', 'row', 'sorry', 'broad', 'create', 'divs', 'tile', 'sql', 'table']]


In [25]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words_train, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words_train], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [26]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


In [28]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_train)

[['apology',
  'sound',
  'noobish',
  'lot',
  'experience',
  'web',
  'development',
  'flash',
  'aside',
  'find',
  'best',
  'solution',
  'project',
  'sql',
  'echo',
  'php',
  'file',
  'echo',
  'border',
  'tr',
  'objectname',
  'url',
  'description',
  'picture',
  'price',
  'location',
  'category',
  'priority',
  'tr',
  'row',
  'echo',
  'tr',
  'echo',
  'row',
  'echo',
  'row',
  'objectname',
  'echo',
  'row',
  'url',
  'echo',
  'row',
  'description',
  'echo',
  'row',
  'picture',
  'echo',
  'row',
  'price',
  'echo',
  'row',
  'location',
  'echo',
  'row',
  'category',
  'echo',
  'row',
  'priority',
  'echo',
  'row',
  'echo',
  'row',
  'echo',
  'tr',
  'echo',
  'pull',
  'whole',
  'sql',
  'user',
  'echo',
  'row',
  'separately',
  'separate',
  'vizualize',
  'echo',
  'tile',
  'isotope',
  'jquery',
  'plug',
  'http',
  'isotope',
  'metafizzy',
  'co',
  'best',
  'dynamically',
  'generate',
  'divs',
  'based',
  'row',
  'fill',
 

In [29]:
data_words_bigrams[0]

['apology',
 'sound',
 'noobish',
 'lot',
 'experience',
 'web',
 'development',
 'flash',
 'aside',
 'find',
 'best',
 'solution',
 'project',
 'sql',
 'echo',
 'php',
 'file',
 'echo',
 'border',
 'tr',
 'objectname',
 'url',
 'description',
 'picture',
 'price',
 'location',
 'category',
 'priority',
 'tr',
 'row',
 'echo',
 'tr',
 'echo',
 'row',
 'echo',
 'row',
 'objectname',
 'echo',
 'row',
 'url',
 'echo',
 'row',
 'description',
 'echo',
 'row',
 'picture',
 'echo',
 'row',
 'price',
 'echo',
 'row',
 'location',
 'echo',
 'row',
 'category',
 'echo',
 'row',
 'priority',
 'echo',
 'row',
 'echo',
 'row',
 'echo',
 'tr',
 'echo',
 'pull',
 'whole',
 'sql',
 'user',
 'echo',
 'row',
 'separately',
 'separate',
 'vizualize',
 'echo',
 'tile',
 'isotope',
 'jquery',
 'plug',
 'http',
 'isotope',
 'metafizzy',
 'co',
 'best',
 'dynamically',
 'generate',
 'divs',
 'based',
 'row',
 'fill',
 'row',
 'sorry',
 'broad',
 'create',
 'divs',
 'tile',
 'sql',
 'table']

In [30]:
trans_lem = WordNetLemmatizer()
def lemmatization(texts):
    texts_out = []
    for sent in texts:
        texts_out.append([trans_lem.lemmatize(word) for word in sent])
    return texts_out

In [31]:
data_lemmatized = lemmatization(data_words_bigrams)

In [32]:
data_lemmatized[:10]

[['apology',
  'sound',
  'noobish',
  'lot',
  'experience',
  'web',
  'development',
  'flash',
  'aside',
  'find',
  'best',
  'solution',
  'project',
  'sql',
  'echo',
  'php',
  'file',
  'echo',
  'border',
  'tr',
  'objectname',
  'url',
  'description',
  'picture',
  'price',
  'location',
  'category',
  'priority',
  'tr',
  'row',
  'echo',
  'tr',
  'echo',
  'row',
  'echo',
  'row',
  'objectname',
  'echo',
  'row',
  'url',
  'echo',
  'row',
  'description',
  'echo',
  'row',
  'picture',
  'echo',
  'row',
  'price',
  'echo',
  'row',
  'location',
  'echo',
  'row',
  'category',
  'echo',
  'row',
  'priority',
  'echo',
  'row',
  'echo',
  'row',
  'echo',
  'tr',
  'echo',
  'pull',
  'whole',
  'sql',
  'user',
  'echo',
  'row',
  'separately',
  'separate',
  'vizualize',
  'echo',
  'tile',
  'isotope',
  'jquery',
  'plug',
  'http',
  'isotope',
  'metafizzy',
  'co',
  'best',
  'dynamically',
  'generate',
  'divs',
  'based',
  'row',
  'fill',
 

In [33]:
import gensim.corpora as corpora
import gensim
# Create Dictionary

# attention: QuantVectorizer et pas tf-IDA


id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 2), (10, 1), (11, 2), (12, 1), (13, 18), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 2)]


In [34]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('apology', 1),
  ('aside', 1),
  ('based', 1),
  ('best', 2),
  ('border', 1),
  ('broad', 1),
  ('category', 2),
  ('co', 1),
  ('create', 1),
  ('description', 2),
  ('development', 1),
  ('divs', 2),
  ('dynamically', 1),
  ('echo', 18),
  ('experience', 1),
  ('file', 1),
  ('fill', 1),
  ('find', 1),
  ('flash', 1),
  ('generate', 1),
  ('http', 1),
  ('isotope', 2),
  ('jquery', 1),
  ('location', 2),
  ('lot', 1),
  ('metafizzy', 1),
  ('noobish', 1),
  ('objectname', 2),
  ('php', 1),
  ('picture', 2),
  ('plug', 1),
  ('price', 2),
  ('priority', 2),
  ('project', 1),
  ('pull', 1),
  ('row', 15),
  ('separate', 1),
  ('separately', 1),
  ('solution', 1),
  ('sorry', 1),
  ('sound', 1),
  ('sql', 3),
  ('table', 1),
  ('tile', 2),
  ('tr', 4),
  ('url', 2),
  ('user', 1),
  ('vizualize', 1),
  ('web', 1),
  ('whole', 1)]]

In [82]:
from pprint import pprint
# number of topics
num_topics = 6
# Build LDA model
lda_model_multicore = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

In [47]:
num_topics = 10
lda_model_standard_10 = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=num_topics)

In [44]:
num_topics = 9
lda_model_standard_9 = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=num_topics)

In [41]:
num_topics = 8
lda_model_standard_8 = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=num_topics)

In [38]:
num_topics = 7
lda_model_standard_7 = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=num_topics)

In [35]:
num_topics = 6
lda_model_standard_6 = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=num_topics)

In [90]:
#5 topics
pprint(lda_model_standard.print_topics())
doc_lda = lda_model_standard[corpus]

[(0,
  '0.020*"id" + 0.018*"class" + 0.014*"function" + 0.012*"text" + 0.012*"type" '
  '+ 0.010*"var" + 0.010*"button" + 0.010*"user" + 0.009*"get" + 0.008*"html"'),
 (1,
  '0.031*"public" + 0.025*"string" + 0.022*"class" + 0.013*"import" + '
  '0.011*"private" + 0.011*"void" + 0.010*"self" + 0.010*"object" + '
  '0.009*"get" + 0.008*"int"'),
 (2,
  '0.014*"image" + 0.011*"server" + 0.011*"user" + 0.010*"http" + '
  '0.010*"request" + 0.009*"app" + 0.008*"get" + 0.007*"code" + '
  '0.007*"message" + 0.007*"client"'),
 (3,
  '0.023*"file" + 0.008*"function" + 0.008*"code" + 0.006*"get" + 0.006*"int" '
  '+ 0.006*"j" + 0.005*"time" + 0.005*"python" + 0.005*"line" + 0.004*"node"'),
 (4,
  '0.090*"java" + 0.041*"android" + 0.014*"com" + 0.014*"jar" + '
  '0.011*"version" + 0.009*"app" + 0.009*"id" + 0.007*"build" + 0.007*"xml" + '
  '0.006*"http"')]


In [39]:
#7 topics
pprint(lda_model_standard_7.print_topics())
doc_lda = lda_model_standard_7[corpus]

[(0,
  '0.010*"string" + 0.010*"int" + 0.008*"get" + 0.008*"code" + 0.008*"list" + '
  '0.007*"time" + 0.007*"function" + 0.007*"array" + 0.007*"object" + '
  '0.007*"query"'),
 (1,
  '0.035*"android" + 0.029*"file" + 0.015*"server" + 0.013*"app" + 0.012*"lib" '
  '+ 0.012*"build" + 0.011*"python" + 0.010*"project" + 0.008*"version" + '
  '0.007*"command"'),
 (2,
  '0.058*"self" + 0.020*"import" + 0.014*"frame" + 0.012*"model" + '
  '0.009*"image" + 0.007*"height" + 0.007*"size" + 0.006*"color" + '
  '0.006*"canvas" + 0.006*"layer"'),
 (3,
  '0.034*"file" + 0.016*"image" + 0.010*"code" + 0.008*"string" + 0.008*"byte" '
  '+ 0.007*"path" + 0.007*"get" + 0.006*"io" + 0.006*"key" + 0.005*"video"'),
 (4,
  '0.021*"public" + 0.017*"user" + 0.014*"string" + 0.013*"get" + '
  '0.011*"class" + 0.011*"request" + 0.010*"app" + 0.009*"http" + 0.008*"j" + '
  '0.008*"function"'),
 (5,
  '0.026*"class" + 0.020*"id" + 0.020*"text" + 0.015*"button" + '
  '0.015*"function" + 0.013*"type" + 0.012*"html

In [36]:
#6 topics
pprint(lda_model_standard_6.print_topics())
doc_lda = lda_model_standard_6[corpus]

[(0,
  '0.015*"row" + 0.012*"string" + 0.012*"get" + 0.012*"set" + 0.011*"id" + '
  '0.010*"select" + 0.010*"text" + 0.009*"key" + 0.008*"query" + 0.008*"null"'),
 (1,
  '0.017*"file" + 0.015*"user" + 0.010*"get" + 0.008*"function" + '
  '0.008*"request" + 0.008*"server" + 0.008*"code" + 0.008*"http" + '
  '0.007*"app" + 0.006*"python"'),
 (2,
  '0.024*"android" + 0.022*"image" + 0.011*"text" + 0.009*"view" + 0.009*"app" '
  '+ 0.009*"button" + 0.007*"j" + 0.007*"function" + 0.007*"id" + '
  '0.007*"width"'),
 (3,
  '0.016*"file" + 0.015*"version" + 0.013*"project" + 0.011*"build" + '
  '0.009*"jar" + 0.008*"server" + 0.008*"app" + 0.007*"java" + 0.007*"web" + '
  '0.006*"module"'),
 (4,
  '0.027*"string" + 0.025*"public" + 0.017*"int" + 0.015*"class" + '
  '0.012*"void" + 0.010*"object" + 0.010*"code" + 0.009*"private" + '
  '0.009*"null" + 0.008*"get"'),
 (5,
  '0.067*"java" + 0.037*"class" + 0.027*"id" + 0.015*"type" + 0.010*"function" '
  '+ 0.010*"input" + 0.008*"button" + 0.008*"

In [42]:
#8 topics
pprint(lda_model_standard_8.print_topics())
doc_lda = lda_model_standard_8[corpus]

[(0,
  '0.026*"file" + 0.023*"app" + 0.019*"project" + 0.015*"version" + '
  '0.015*"build" + 0.010*"php" + 0.008*"com" + 0.007*"plugin" + 0.006*"http" + '
  '0.005*"dependency"'),
 (1,
  '0.109*"java" + 0.016*"jar" + 0.016*"class" + 0.015*"import" + 0.013*"xml" + '
  '0.012*"web" + 0.010*"http" + 0.009*"public" + 0.009*"file" + 0.008*"final"'),
 (2,
  '0.020*"user" + 0.016*"get" + 0.013*"string" + 0.013*"public" + '
  '0.012*"request" + 0.011*"object" + 0.010*"class" + 0.008*"code" + '
  '0.008*"set" + 0.008*"http"'),
 (3,
  '0.035*"string" + 0.020*"public" + 0.016*"id" + 0.014*"row" + 0.013*"query" '
  '+ 0.012*"int" + 0.012*"select" + 0.012*"self" + 0.012*"date" + '
  '0.012*"null"'),
 (4,
  '0.024*"class" + 0.021*"function" + 0.016*"text" + 0.016*"id" + 0.014*"type" '
  '+ 0.013*"var" + 0.013*"button" + 0.013*"html" + 0.011*"j" + 0.009*"input"'),
 (5,
  '0.017*"window" + 0.016*"file" + 0.014*"server" + 0.014*"lib" + '
  '0.012*"python" + 0.007*"library" + 0.007*"command" + 0.007*"l

In [45]:
#9 topics
pprint(lda_model_standard_9.print_topics())
doc_lda = lda_model_standard_9[corpus]

[(0,
  '0.066*"android" + 0.026*"view" + 0.018*"id" + 0.014*"import" + '
  '0.014*"public" + 0.013*"app" + 0.012*"string" + 0.010*"com" + 0.009*"void" '
  '+ 0.009*"button"'),
 (1,
  '0.031*"user" + 0.018*"request" + 0.014*"http" + 0.013*"client" + '
  '0.013*"get" + 0.013*"service" + 0.012*"response" + 0.012*"message" + '
  '0.011*"api" + 0.011*"server"'),
 (2,
  '0.037*"image" + 0.014*"width" + 0.011*"height" + 0.010*"row" + '
  '0.009*"color" + 0.009*"size" + 0.008*"cell" + 0.008*"code" + 0.007*"frame" '
  '+ 0.007*"grid"'),
 (3,
  '0.040*"string" + 0.034*"public" + 0.026*"class" + 0.022*"int" + '
  '0.019*"object" + 0.015*"get" + 0.014*"void" + 0.011*"private" + '
  '0.011*"code" + 0.011*"list"'),
 (4,
  '0.130*"java" + 0.034*"self" + 0.021*"jar" + 0.014*"class" + 0.012*"com" + '
  '0.011*"web" + 0.010*"version" + 0.008*"http" + 0.008*"final" + '
  '0.008*"lang"'),
 (5,
  '0.028*"file" + 0.018*"app" + 0.012*"project" + 0.012*"build" + 0.011*"lib" '
  '+ 0.010*"server" + 0.010*"pyth

In [48]:
#10 topics
pprint(lda_model_standard_10.print_topics())
doc_lda = lda_model_standard_10[corpus]

[(0,
  '0.158*"java" + 0.024*"jar" + 0.014*"com" + 0.011*"sql" + 0.009*"lang" + '
  '0.009*"core" + 0.009*"class" + 0.008*"final" + 0.008*"thread" + '
  '0.007*"oracle"'),
 (1,
  '0.042*"public" + 0.039*"string" + 0.022*"class" + 0.015*"private" + '
  '0.013*"service" + 0.013*"get" + 0.013*"web" + 0.011*"http" + 0.011*"void" + '
  '0.010*"system"'),
 (2,
  '0.029*"file" + 0.016*"user" + 0.010*"code" + 0.009*"server" + 0.009*"get" + '
  '0.008*"http" + 0.007*"app" + 0.006*"time" + 0.005*"request" + '
  '0.005*"window"'),
 (3,
  '0.054*"android" + 0.028*"int" + 0.014*"app" + 0.012*"void" + 0.010*"id" + '
  '0.010*"view" + 0.008*"com" + 0.008*"code" + 0.008*"include" + 0.007*"std"'),
 (4,
  '0.044*"j" + 0.034*"version" + 0.024*"project" + 0.023*"module" + '
  '0.019*"import" + 0.018*"plugin" + 0.013*"build" + 0.012*"dependency" + '
  '0.012*"src" + 0.010*"maven"'),
 (5,
  '0.012*"model" + 0.010*"window" + 0.009*"function" + 0.008*"add" + '
  '0.008*"code" + 0.007*"time" + 0.007*"set" + 0.

In [37]:
# Compute Perplexity for 6 topics
print('\nPerplexity: ', lda_model_standard_6.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model_standard_6, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.221572309918148

Coherence Score:  0.5023480017054885


In [40]:
# Compute Perplexity for 7 topics
print('\nPerplexity: ', lda_model_standard_7.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model_standard_7, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.346990427204569

Coherence Score:  0.5258252067989905


In [43]:
# Compute Perplexity for 8 topics
print('\nPerplexity: ', lda_model_standard_8.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model_standard_8, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.557128143691095

Coherence Score:  0.5319131910196178


In [46]:
# Compute Perplexity for 9 topics
print('\nPerplexity: ', lda_model_standard_9.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model_standard_9, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.820415245379102

Coherence Score:  0.5415668304585756


In [49]:
# Compute Perplexity for 10 topics
print('\nPerplexity: ', lda_model_standard_10.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model_standard_10, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.145048905518536

Coherence Score:  0.519814829723938


In [91]:
# Compute Perplexity for 5 topics
print('\nPerplexity: ', lda_model_standard.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model_standard, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.1032055476979

Coherence Score:  0.4937728480781467


In [75]:
#6 topics
import pyLDAvis.gensim_models
import pickle
import pyLDAvis
import os

# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model_standard_6, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/maurelco/opt/anaconda3

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.095579  0.026955       1        1  29.203740
4     -0.026723  0.138243       2        1  18.376472
2     -0.054237 -0.177798       3        1  14.876797
3      0.232076 -0.007458       4        1  14.861626
5     -0.104639 -0.105434       5        1  12.859647
0     -0.142056  0.125492       6        1   9.821718, topic_info=         Term          Freq         Total Category  logprob  loglift
80       java  37790.000000  37790.000000  Default  30.0000  30.0000
53      class  34589.000000  34589.000000  Default  29.0000  29.0000
294    string  27799.000000  27799.000000  Default  28.0000  28.0000
255     image  14401.000000  14401.000000  Default  27.0000  27.0000
2348  android  16823.000000  16823.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
937      date   1432.860154   5586.474456   Topic6  -5.5662   0.9599
46       user   1808.921331  23295.386368   Topic6  -5.3332  -0.2350
181     model   1488.844709   8217.150803   Topic6  -5.5279   0.6124
630      type   1530.147113  18878.516227   Topic6  -5.5006  -0.1921
53      class   1499.369441  34589.908183   Topic6  -5.5209  -0.8179

[568 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
16470      6  0.994224             aaa
6534       1  0.998163  access_control
6535       1  0.997008    access_token
6535       4  0.001511    access_token
125        1  0.211835             add
...      ...       ...             ...
2368       5  0.416021             xml
2368       6  0.005719             xml
20851      2  0.993193          xrange
30245      3  0.990980   youtube_video
8820       5  0.993617      zygoteinit

[1705 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 3, 4, 6, 1])

In [76]:
#7 topics
import pyLDAvis.gensim_models
import pickle
import pyLDAvis
import os

# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model_standard_7, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/setuptools/_dist

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.044298  0.111650       1        1  23.948355
0     -0.056907  0.111738       2        1  23.439157
5     -0.144739  0.111248       3        1  15.328155
1      0.151032 -0.009369       4        1  11.932904
3     -0.003192  0.049581       5        1  11.867403
6      0.241298 -0.143309       6        1   8.839267
2     -0.231790 -0.231539       7        1   4.644758, topic_info=         Term          Freq         Total Category  logprob  loglift
80       java  37638.000000  37638.000000  Default  30.0000  30.0000
15       file  34030.000000  34030.000000  Default  29.0000  29.0000
2348  android  17472.000000  17472.000000  Default  28.0000  28.0000
1558     self  11323.000000  11323.000000  Default  27.0000  27.0000
53      class  34848.000000  34848.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
1370    label    820.324778   6289.081606   Topic7  -5.3751   1.0326
1186   screen    618.402874   2225.953858   Topic7  -5.6577   1.7886
228      code    810.312923  25236.415587   Topic7  -5.3874  -0.3692
338      view    635.023659   9598.792757   Topic7  -5.6312   0.3537
1142   python    528.744384   7517.567236   Topic7  -5.8143   0.4149

[627 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
6534       1  0.997620  access_control
6535       1  0.997191    access_token
6535       5  0.001488    access_token
6245       1  0.851097         account
6245       2  0.094627         account
...      ...       ...             ...
804        5  0.003447             xsd
804        6  0.895453             xsd
17798      7  0.994210         y_train
4621       1  0.993323             yii
4621       5  0.004536             yii

[2057 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 6, 2, 4, 7, 3])

In [52]:
#8 topics
import pyLDAvis.gensim_models
import pickle
import pyLDAvis
import os

# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model_standard_8, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/setuptools/_dist

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.042524  0.068944       1        1  21.158964
6      0.065930  0.160492       2        1  19.070368
4     -0.086280  0.130029       3        1  16.959392
3     -0.098722  0.078157       4        1  10.260935
5      0.198103 -0.018653       5        1   9.386706
0      0.130169 -0.076991       6        1   9.204661
1      0.011974 -0.257698       7        1   8.367977
7     -0.263699 -0.084280       8        1   5.590997, topic_info=         Term          Freq         Total Category  logprob  loglift
80       java  37980.000000  37980.000000  Default  30.0000  30.0000
2348  android  16995.000000  16995.000000  Default  29.0000  29.0000
294    string  28677.000000  28677.000000  Default  28.0000  28.0000
284    public  24236.000000  24236.000000  Default  27.0000  27.0000
15       file  34210.000000  34210.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
570      null   1273.917105  11900.225252   Topic8  -5.1204   0.6496
294    string   1473.241435  28677.926251   Topic8  -4.9750  -0.0847
53      class   1419.745279  35102.833068   Topic8  -5.0120  -0.3238
258       int   1045.980321  14250.824341   Topic8  -5.3175   0.2722
619      text    935.158960  15938.533157   Topic8  -5.4295   0.0482

[699 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
16470      1  0.995243             aaa
27987      6  0.989071             acc
6534       1  0.949482  access_control
6534       3  0.014087  access_control
6534       4  0.033809  access_control
...      ...       ...             ...
4537       4  0.055861             zip
4537       5  0.006572             zip
4537       6  0.697716             zip
4537       7  0.028478             zip
8820       8  0.995759      zygoteinit

[2465 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 7, 5, 4, 6, 1, 2, 8])

In [50]:
#9 topics
import pyLDAvis.gensim_models
import pickle
import pyLDAvis
import os

# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model_standard_9, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/maurelco/opt/anaconda3

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.140836 -0.078190       1        1  16.253952
7      0.073686  0.040962       2        1  14.760840
3      0.051111  0.063260       3        1  13.856012
5     -0.066801 -0.194430       4        1  13.816252
1     -0.092250 -0.061001       5        1  13.035528
4     -0.298255  0.049708       6        1   7.452319
2      0.197063  0.058977       7        1   7.131390
8      0.024660 -0.128389       8        1   6.854296
0     -0.030049  0.249103       9        1   6.839410, topic_info=         Term          Freq         Total Category  logprob  loglift
80       java  39306.000000  39306.000000  Default  30.0000  30.0000
2348  android  17788.000000  17788.000000  Default  29.0000  29.0000
15       file  34535.000000  34535.000000  Default  28.0000  28.0000
294    string  29289.000000  29289.000000  Default  27.0000  27.0000
284    public  25211.000000  25211.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
228      code   1312.102929  26108.333786   Topic9  -5.2924  -0.3082
53      class   1342.204234  36011.184739   Topic9  -5.2697  -0.6070
255     image   1095.750942  14682.903427   Topic9  -5.4726   0.0872
258       int   1081.621694  14481.357930   Topic9  -5.4856   0.0881
351       get    981.779882  28488.704549   Topic9  -5.5824  -0.6854

[816 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
8553       1  0.065458             ac
8553       2  0.011901             ac
8553       3  0.011901             ac
8553       4  0.117031             ac
8553       5  0.033721             ac
...      ...       ...            ...
17798      2  0.993330        y_train
30245      8  0.992637  youtube_video
2982       3  0.989838        zone_js
8820       6  0.003733     zygoteinit
8820       9  0.992932     zygoteinit

[3145 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 8, 4, 6, 2, 5, 3, 9, 1])

In [51]:
#10 topics
import pyLDAvis.gensim_models
import pickle
import pyLDAvis
import os

# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model_standard_10, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/setuptools/_dist

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.054282  0.027713       1        1  22.577056
6      0.142001 -0.076461       2        1  17.229143
1     -0.042608 -0.120693       3        1  13.845771
7      0.176637 -0.076458       4        1   9.845174
3      0.034537  0.020200       5        1   8.661361
5      0.151009  0.006303       6        1   8.023297
0     -0.262353 -0.193492       7        1   5.991805
9     -0.126874  0.182608       8        1   5.903482
8      0.011231  0.135611       9        1   4.338183
4     -0.137863  0.094669      10        1   3.584728, topic_info=         Term          Freq         Total Category  logprob  loglift
80       java  38444.000000  38444.000000  Default  30.0000  30.0000
2348  android  17775.000000  17775.000000  Default  29.0000  29.0000
284    public  25310.000000  25310.000000  Default  28.0000  28.0000
294    string  29287.000000  29287.000000  Default  27.0000  27.0000
857         j   9872.000000   9872.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
886   package    678.636007   4783.038930  Topic10  -5.3057   1.3757
986    target    579.351432   3158.359371  Topic10  -5.4638   1.6326
654       app    683.837176  19124.538192  Topic10  -5.2980  -0.0025
55     config    554.236198   4372.439022  Topic10  -5.5082   1.2630
125       add    599.823624  12036.603040  Topic10  -5.4291   0.3294

[865 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
17003      1  0.994321            aad
27987      8  0.995522            acc
2536       1  0.640155         access
2536       2  0.085690         access
2536       3  0.132735         access
...      ...       ...            ...
3636       3  0.008728        youtube
3636       5  0.008728        youtube
3636       6  0.002182        youtube
3636      10  0.002182        youtube
30245      1  0.993320  youtube_video

[3405 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 7, 2, 8, 4, 6, 1, 10, 9, 5])

In [ ]:
# coherence score est utilise d'habitude, pas forcement la mesure la plus pertinente, veille a utiliser des mesures plus orientes metiers - combinaison d'approche, le taux de couverture

In [53]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("/Users/maurelco/Developer/Python/Projet 4/models/model_9")
lda_model_standard_9.save(temp_file)

#Load a potentially pretrained model from disk.
lda_model_9 = gensim.models.ldamodel.LdaModel.load(temp_file)

In [54]:
data_words_test = list(sent_to_words(list_title_body_test))

print(data_words_test[:1])

[['small', 'writing', 'unit', 'django', 'is_valid', 'seen', 'code', 'day', 'reading', 'google', 'yet', 'find', 'looking', 'code', 'forms', 'py', 'test_forms', 'py', 'forms', 'py', 'class', 'dataselectform', 'forms', 'form', 'these', 'init', 'funct', 'result_type', 'forms', 'choicefield', 'widget', 'forms', 'select', 'attrs', 'class', 'field', 'long', 'band_selection', 'forms', 'widget', 'forms', 'selectmultiple', 'attrs', 'class', 'multiselect', 'field', 'long', 'title', 'forms', 'charfield', 'widget', 'forms', 'hiddeninput', 'description', 'forms', 'charfield', 'widget', 'forms', 'hiddeninput', 'self', 'result_list', 'band_list', 'args', 'kwargs', 'super', 'dataselectform', 'self', 'args', 'kwargs', 'result_list', 'band_list', 'self', 'fields', 'result_type', 'forms', 'choicefield', 'choice', 'result_list', 'widget', 'forms', 'select', 'attrs', 'class', 'field', 'long', 'self', 'fields', 'band_selection', 'forms', 'widget', 'forms', 'selectmultiple', 'attrs', 'class', 'multiselect', '

In [55]:
# Build the bigram and trigram models
bigram_test = gensim.models.Phrases(data_words_test, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram_test = gensim.models.Phrases(bigram_test[data_words_test], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram_test)
trigram_mod = gensim.models.phrases.Phraser(trigram_test)

In [56]:
# Form Bigrams
data_words_bigrams_test = make_bigrams(data_words_test)
data_words_bigrams_test

[['small',
  'writing',
  'unit',
  'django',
  'is_valid',
  'seen',
  'code',
  'day',
  'reading',
  'google',
  'yet',
  'find',
  'looking',
  'code',
  'forms',
  'py',
  'test_forms',
  'py',
  'forms',
  'py',
  'class',
  'dataselectform',
  'forms',
  'form',
  'these',
  'init',
  'funct',
  'result_type',
  'forms',
  'choicefield',
  'widget_forms',
  'select',
  'attrs',
  'class',
  'field',
  'long',
  'band_selection',
  'forms',
  'widget_forms',
  'selectmultiple',
  'attrs',
  'class',
  'multiselect',
  'field',
  'long',
  'title',
  'forms_charfield',
  'widget_forms',
  'hiddeninput',
  'description',
  'forms_charfield',
  'widget_forms',
  'hiddeninput',
  'self',
  'result_list',
  'band_list',
  'args_kwargs',
  'super',
  'dataselectform',
  'self',
  'args_kwargs',
  'result_list',
  'band_list',
  'self',
  'fields',
  'result_type',
  'forms',
  'choicefield',
  'choice',
  'result_list',
  'widget_forms',
  'select',
  'attrs',
  'class',
  'field',
  '

In [57]:
data_words_bigrams_test[0]

['small',
 'writing',
 'unit',
 'django',
 'is_valid',
 'seen',
 'code',
 'day',
 'reading',
 'google',
 'yet',
 'find',
 'looking',
 'code',
 'forms',
 'py',
 'test_forms',
 'py',
 'forms',
 'py',
 'class',
 'dataselectform',
 'forms',
 'form',
 'these',
 'init',
 'funct',
 'result_type',
 'forms',
 'choicefield',
 'widget_forms',
 'select',
 'attrs',
 'class',
 'field',
 'long',
 'band_selection',
 'forms',
 'widget_forms',
 'selectmultiple',
 'attrs',
 'class',
 'multiselect',
 'field',
 'long',
 'title',
 'forms_charfield',
 'widget_forms',
 'hiddeninput',
 'description',
 'forms_charfield',
 'widget_forms',
 'hiddeninput',
 'self',
 'result_list',
 'band_list',
 'args_kwargs',
 'super',
 'dataselectform',
 'self',
 'args_kwargs',
 'result_list',
 'band_list',
 'self',
 'fields',
 'result_type',
 'forms',
 'choicefield',
 'choice',
 'result_list',
 'widget_forms',
 'select',
 'attrs',
 'class',
 'field',
 'long',
 'self',
 'fields',
 'band_selection',
 'forms',
 'widget_forms',
 's

In [58]:
data_lemmatized_test = lemmatization(data_words_bigrams_test)

In [59]:
data_lemmatized_test[:1]

[['small',
  'writing',
  'unit',
  'django',
  'is_valid',
  'seen',
  'code',
  'day',
  'reading',
  'google',
  'yet',
  'find',
  'looking',
  'code',
  'form',
  'py',
  'test_forms',
  'py',
  'form',
  'py',
  'class',
  'dataselectform',
  'form',
  'form',
  'these',
  'init',
  'funct',
  'result_type',
  'form',
  'choicefield',
  'widget_forms',
  'select',
  'attrs',
  'class',
  'field',
  'long',
  'band_selection',
  'form',
  'widget_forms',
  'selectmultiple',
  'attrs',
  'class',
  'multiselect',
  'field',
  'long',
  'title',
  'forms_charfield',
  'widget_forms',
  'hiddeninput',
  'description',
  'forms_charfield',
  'widget_forms',
  'hiddeninput',
  'self',
  'result_list',
  'band_list',
  'args_kwargs',
  'super',
  'dataselectform',
  'self',
  'args_kwargs',
  'result_list',
  'band_list',
  'self',
  'field',
  'result_type',
  'form',
  'choicefield',
  'choice',
  'result_list',
  'widget_forms',
  'select',
  'attrs',
  'class',
  'field',
  'long',


In [269]:
range(len(data_lemmatized_test_rejoin))

range(0, 15000)

In [ ]:
data_lemmatized_test_rejoin = data_lemmatized_test.copy()

In [272]:
" ".join(data_lemmatized_test_rejoin)

TypeError: sequence item 0: expected str instance, list found

In [274]:
data_lemmatized_test_rejoin = data_lemmatized_test.copy()
for i in range(len(data_lemmatized_test_rejoin)):
    for value in data_lemmatized_test_rejoin[i]:
        data_lemmatized_test_rejoin[i] = " ".join(value)
data_lemmatized_test_rejoin

KeyboardInterrupt: 

In [60]:
# Create a new corpus, made of previously unseen documents.
other_texts = data_lemmatized_test
# Term Document Frequency
other_corpus = [id2word.doc2bow(text) for text in other_texts]

In [61]:
other_texts[:2]

[['small',
  'writing',
  'unit',
  'django',
  'is_valid',
  'seen',
  'code',
  'day',
  'reading',
  'google',
  'yet',
  'find',
  'looking',
  'code',
  'form',
  'py',
  'test_forms',
  'py',
  'form',
  'py',
  'class',
  'dataselectform',
  'form',
  'form',
  'these',
  'init',
  'funct',
  'result_type',
  'form',
  'choicefield',
  'widget_forms',
  'select',
  'attrs',
  'class',
  'field',
  'long',
  'band_selection',
  'form',
  'widget_forms',
  'selectmultiple',
  'attrs',
  'class',
  'multiselect',
  'field',
  'long',
  'title',
  'forms_charfield',
  'widget_forms',
  'hiddeninput',
  'description',
  'forms_charfield',
  'widget_forms',
  'hiddeninput',
  'self',
  'result_list',
  'band_list',
  'args_kwargs',
  'super',
  'dataselectform',
  'self',
  'args_kwargs',
  'result_list',
  'band_list',
  'self',
  'field',
  'result_type',
  'form',
  'choicefield',
  'choice',
  'result_list',
  'widget_forms',
  'select',
  'attrs',
  'class',
  'field',
  'long',


In [62]:
other_corpus[:2]

[[(9, 4),
  (17, 1),
  (53, 5),
  (189, 1),
  (228, 2),
  (351, 1),
  (377, 1),
  (402, 1),
  (529, 1),
  (551, 1),
  (746, 1),
  (895, 1),
  (992, 1),
  (998, 1),
  (1013, 1),
  (1025, 1),
  (1029, 2),
  (1030, 1),
  (1055, 1),
  (1149, 1),
  (1164, 4),
  (1247, 4),
  (1420, 6),
  (1446, 1),
  (1496, 1),
  (1502, 10),
  (1546, 1),
  (1558, 6),
  (1676, 4),
  (1725, 2),
  (1756, 4),
  (1838, 1),
  (2642, 1),
  (3063, 1),
  (3265, 2),
  (3765, 2),
  (4447, 2),
  (10896, 4),
  (14930, 4),
  (16050, 2),
  (16057, 3),
  (22761, 6),
  (23682, 2),
  (24274, 2),
  (26367, 3),
  (27288, 2),
  (28844, 1),
  (41046, 1),
  (42320, 1),
  (64138, 1),
  (83003, 3),
  (112563, 1),
  (160283, 2)],
 [(3, 1),
  (49, 1),
  (68, 1),
  (183, 2),
  (189, 1),
  (274, 1),
  (338, 3),
  (359, 3),
  (486, 1),
  (816, 1),
  (924, 1),
  (1083, 1),
  (1093, 4),
  (1147, 1),
  (1173, 1),
  (1435, 1),
  (1905, 3),
  (2039, 1),
  (2182, 1),
  (2432, 1),
  (2815, 1),
  (3868, 1),
  (3887, 1),
  (5043, 1),
  (5673, 1),

In [63]:
lda_output_transform_test = lda_model_standard_9[other_corpus]

In [64]:
def create_initial_lda_df(lda_output):
    df_topic = pd.DataFrame(lda_output)
    for doc in range(len(lda_output)):
        for tup in lda_output[doc]:
            df_topic[tup[0]][doc] = tup[1]

    return df_topic

In [65]:
df_topic = create_initial_lda_df(lda_output_transform_test)

In [66]:
def clean_columns_from_tuples(df,topic_int_column):
    for i in range(len(df[topic_int_column])):
        if type(df[topic_int_column][i]) != np.float32:
            df[topic_int_column][i] = 0

    return df

In [68]:
clean_columns_from_tuples(df_topic,0)
clean_columns_from_tuples(df_topic,1)
clean_columns_from_tuples(df_topic,2)
clean_columns_from_tuples(df_topic,3)
clean_columns_from_tuples(df_topic,4)
clean_columns_from_tuples(df_topic,5)
clean_columns_from_tuples(df_topic,6)
clean_columns_from_tuples(df_topic,7)
clean_columns_from_tuples(df_topic,8)

,0,1,2,3,4,5,6,7,8
0,0,0.042948,0,0.189585,0.115442,0.080536,0,0.567499,0
1,0.086985,0,0.177345,0,0.03435,0.316441,0.3754,0,0
2,0.017619,0.256898,0.03138,0,0,0.150843,0.164294,0.183876,0.187156
3,0,0.075181,0.066565,0.069003,0.05073,0.074037,0,0.661305,0
4,0.011126,0.011128,0.011124,0.494811,0.427305,0.011126,0.01113,0.011125,0.011125
...,...,...,...,...,...,...,...,...,...
14995,0,0,0.446206,0,0,0.299788,0,0.222472,0
14996,0.144615,0.066113,0.248743,0.374725,0,0,0.014068,0,0.150424
14997,0,0,0.256652,0.09735,0,0,0.327556,0.27987,0.034769
14998,0,0.546047,0.048555,0.17883,0,0.028095,0.189186,0,0


In [70]:
df_topic

,0,1,2,3,4,5,6,7,8
0,0,0.042948,0,0.189585,0.115442,0.080536,0,0.567499,0
1,0.086985,0,0.177345,0,0.03435,0.316441,0.3754,0,0
2,0.017619,0.256898,0.03138,0,0,0.150843,0.164294,0.183876,0.187156
3,0,0.075181,0.066565,0.069003,0.05073,0.074037,0,0.661305,0
4,0.011126,0.011128,0.011124,0.494811,0.427305,0.011126,0.01113,0.011125,0.011125
...,...,...,...,...,...,...,...,...,...
14995,0,0,0.446206,0,0,0.299788,0,0.222472,0
14996,0.144615,0.066113,0.248743,0.374725,0,0,0.014068,0,0.150424
14997,0,0,0.256652,0.09735,0,0,0.327556,0.27987,0.034769
14998,0,0.546047,0.048555,0.17883,0,0.028095,0.189186,0,0


In [69]:
type(df_topic)

pandas.core.frame.DataFrame

In [71]:
# column names
topicnames = ["Topic" + str(i) for i in range(6)]

# index names
docnames = ["Doc" + str(i) for i in range(len(other_texts))]

# Get dominant topic for each document
dominant_topic = np.argmax(df_topic.values, axis=1)
df_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_topic.head(20).style.applymap(color_green).applymap(make_bold)
df_document_topics

,0,1,2,3,4,5,6,7,8,dominant_topic
0,0,0.042948,0,0.189585,0.115442,0.080536,0,0.567499,0,7
1,0.086985,0,0.177345,0,0.034350,0.316441,0.375400,0,0,6
2,0.017619,0.256898,0.031380,0,0,0.150843,0.164294,0.183876,0.187156,1
3,0,0.075181,0.066565,0.069003,0.050730,0.074037,0,0.661305,0,7
4,0.011126,0.011128,0.011124,0.494811,0.427305,0.011126,0.011130,0.011125,0.011125,3
5,0,0,0.102839,0,0,0.129323,0,0.512178,0.251923,7
6,0,0.028294,0,0,0,0.530493,0.275191,0.148433,0.012830,5
7,0,0.505722,0,0.118009,0.104617,0.031192,0.139975,0.096947,0,1
8,0.523857,0,0.037276,0.182573,0,0.082596,0,0,0.167631,0
9,0,0.206752,0,0.140362,0.021256,0,0,0.628234,0,7


In [72]:
df_topic_distribution = df_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,6,3448
1,5,2439
2,7,2250
3,3,2012
4,1,1893
5,2,1082
6,8,806
7,0,702
8,4,368


In [73]:
array_proba_words_per_topics = lda_model_standard_9.get_topics()

In [74]:
array_proba_words_per_topics[:30]

array([[6.0769247e-07, 9.7538361e-07, 2.0153967e-04, ..., 5.0908551e-07,
        5.6527062e-07, 5.0908551e-07],
       [2.1858283e-05, 2.6056784e-07, 7.2531047e-04, ..., 2.5431021e-07,
        2.6941808e-07, 2.5431021e-07],
       [6.2017396e-05, 5.5983608e-05, 8.9347898e-04, ..., 4.5132063e-07,
        4.5132077e-07, 4.5132063e-07],
       ...,
       [8.3168728e-05, 3.9220064e-05, 2.5026258e-03, ..., 2.0254069e-07,
        2.0440098e-07, 2.0254069e-07],
       [8.7590779e-06, 2.4320867e-05, 5.4873974e-04, ..., 2.1386239e-07,
        2.1386242e-07, 2.1386239e-07],
       [4.8481867e-05, 1.8392658e-06, 4.5202661e-04, ..., 4.7141660e-07,
        4.7141677e-07, 4.7141660e-07]], dtype=float32)

In [ ]:
#Pour trouver les probabilités de mots pour les questions Test, il suffit de reprendre la même formule, sachant que M(test)quest-topics est le résultat du predict du LDA sur les questions Test.

In [77]:
m_test_quest_topics = df_topic.drop('dominant_topic', axis=1)
m_test_quest_topics

,0,1,2,3,4,5,6,7,8
0,0,0.042948,0,0.189585,0.115442,0.080536,0,0.567499,0
1,0.086985,0,0.177345,0,0.03435,0.316441,0.3754,0,0
2,0.017619,0.256898,0.03138,0,0,0.150843,0.164294,0.183876,0.187156
3,0,0.075181,0.066565,0.069003,0.05073,0.074037,0,0.661305,0
4,0.011126,0.011128,0.011124,0.494811,0.427305,0.011126,0.01113,0.011125,0.011125
...,...,...,...,...,...,...,...,...,...
14995,0,0,0.446206,0,0,0.299788,0,0.222472,0
14996,0.144615,0.066113,0.248743,0.374725,0,0,0.014068,0,0.150424
14997,0,0,0.256652,0.09735,0,0,0.327556,0.27987,0.034769
14998,0,0.546047,0.048555,0.17883,0,0.028095,0.189186,0,0


In [78]:
type(m_test_quest_topics)

pandas.core.frame.DataFrame

In [79]:
array_proba_topics_per_doc = m_test_quest_topics.to_numpy()
type(array_proba_topics_per_doc)

numpy.ndarray

In [83]:
print(array_proba_topics_per_doc[:2])

[[0 0.042948425 0 0.18958504 0.115442045 0.080536224 0 0.56749856 0]
 [0.08698544 0 0.17734452 0 0.03435007 0.3164406 0.3754004 0 0]]


In [ ]:
print(array_proba_words_per_topics[:2])

In [80]:
array_proba_words_per_doc = np.tensordot(array_proba_topics_per_doc, array_proba_words_per_topics, 2)
array_proba_words_per_doc.head(30)

ValueError: shape-mismatch for sum

In [ ]:
[[sum(a*b for a,b in zip(A_row, B_col)) for B_col in zip(*B)]
 for A_row in A]

In [1]:
type(m_test_quest_topics)

NameError: name 'm_test_quest_topics' is not defined

In [ ]:
array_proba_words_per_doc = np.matmul(array_proba_topics_per_doc,array_proba_words_per_topics)
array_proba_words_per_doc[:30]

In [ ]:
array_proba_words_per_doc.to_csv('/Users/maurelco/Developer/Python/Projet 4/data/Cleaned/matrice_proba_words_per_doc.csv')

In [267]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model_standard_6.get_topics())

# Assign Column and Index
df_topic_keywords.columns = vectorizer_1.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


ValueError: Length mismatch: Expected axis has 186618 elements, new values have 46815 elements

In [158]:
df_document_topic = pd.DataFrame(lda_output, columns=topicnames, index=docnames)

In [ ]:
#aujourd'hui on a des Transformers , tres gros reseaux, avec des millions de parametres -

### 3. Perform TF-IDF transformation

In [ ]:
'quest ce qui rendra plus facile ton implementation - le fait de fixer ca dans mes colonnes'

In [ ]:
tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(dataset)
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print (df.head(25))